In [ ]:
from yugiquery import *

init_notebook_mode(all_interactive=True)

header("My Decks")

---

# Data loading

In [ ]:
_ = git.ensure_repo()

## Read decks

In [ ]:
# Timestamp
timestamp = arrow.utcnow()

In [ ]:
# Load decks from YDK and decklist files
deck_df = pd.concat([get_ydk(), get_decklists()], ignore_index=True)

In [ ]:
# Process the deck data frame
deck_df = find_cards(deck_df, merge_data=True)

In [ ]:
deck_df

In [ ]:
# Get latest file if exist
previous_df, previous_ts = load_latest_data("deck")

if previous_df is not None:
    previous_df = previous_df.astype(deck_df[previous_df.columns.intersection(deck_df.columns)].dtypes.to_dict())

In [ ]:
if previous_df is None:
    deck_changelog = None
    print("Skipped")
else:
    deck_changelog = generate_changelog(previous_df, deck_df, col="Name")
    if not deck_changelog.empty:
        display(deck_changelog)
        deck_changelog.to_csv(
            dirs.DATA
            / make_filename(
                report="deck",
                timestamp=timestamp,
                previous_timestamp=previous_ts,
            ),
            index=True,
        )
        print("Changelog saved")

In [ ]:
if deck_changelog is not None and deck_changelog.empty:
    print("No changes. New data not saved")
else:
    deck_df.to_csv(
        dirs.DATA / make_filename(report="deck", timestamp=timestamp),
        index=False,
    )
    print("Data saved")

In [ ]:
deck_df

In [ ]:
# Other

# Merge the collection and deck data frames
collection_df = get_collection()
if collection_df is not None:
    collection_df = assign_deck(collection_df, deck_df=deck_df, return_collection=False)

In [ ]:
collection_df

In [ ]:
temp = deck_df.groupby(["Deck", "Card type"])["Count"].sum().unstack(0)

In [ ]:
temp

In [ ]:
colors = [plot.colors_dict[type] for type in temp.index]
fig, axes = plt.subplots(4, 3, figsize=(16, 20))
axes = axes.flatten()
fig.suptitle("Card Type Distribution", fontsize=20)
for i, deck in enumerate(temp):
    axes[i].pie(
        temp[deck],
        autopct="%1.1f%%",
        startangle=90,
        wedgeprops=dict(width=0.4),
        pctdistance=0.8,
        colors=colors,
        counterclock=False,
    )
    axes[i].set_title(deck, y=0.5, x=0.5, fontsize=16, pad=-6)

for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

fig.legend(temp.index, loc="upper center", fontsize=16, ncol=len(temp.index), bbox_to_anchor=(0.5, 0.95))
fig.tight_layout(rect=[0, 0, 1, 0.95])

In [ ]:
ax = temp.plot.pie(subplots=True, autopct="%1.1f%%", figsize=(16, 4), title="Card type distribution")
for a in ax:
    a.set_ylabel(a.get_ylabel(), fontsize=16)

plt.tight_layout()
plt.show()

In [ ]:
temp = deck_df.groupby(["Deck", "Monster type"])["Count"].sum().unstack(0)
temp = temp[temp.notna().all(axis=1)]
ax = temp.plot.pie(subplots=True, autopct="%1.1f%%", figsize=(16, 4), title="Card type distribution")
for a in ax:
    a.set_ylabel(a.get_ylabel(), fontsize=16)

plt.tight_layout()
plt.show()